In [3]:
# %load transfer_learning.py
from keras.preprocessing.image import img_to_array,load_img
from keras.models import *
from keras.layers import *
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.optimizers import Adam,SGD
import glob

class dataProcess(object):
    def __init__(self, out_rows, out_cols, data_path="./orig_data/image", label_path="./orig_data/mask",
                 test_path="./orig_data/test", npy_path="./npy_data", img_type="png"):
        # 数据处理类，初始化
        self.out_rows = out_rows
        self.out_cols = out_cols
        self.data_path = data_path
        self.label_path = label_path
        self.img_type = img_type
        self.test_path = test_path
        self.npy_path = npy_path

    # 创建训练数据
    def create_train_data(self):
        i = 0
        print('-' * 30)
        print('Creating training images...')
        print('-' * 30)
        imgs = glob.glob(self.data_path + "/*." + self.img_type)
        img_len=len(imgs)
        print(len(imgs))

		# 此处有改动，1变为3
        imgdatas = np.ndarray((len(imgs), self.out_rows, self.out_cols, 3), dtype=np.uint8)
        imglabels = np.ndarray((len(imgs), self.out_rows, self.out_cols, 1), dtype=np.uint8)
        for imgname in imgs:
            midname = imgname[imgname.rindex("\\") + 1:]
            img = load_img(self.data_path + "/" + midname, color_mode='rgb')
            label = load_img(self.label_path + "/" + midname, grayscale=True)
            img = img_to_array(img)
            label = img_to_array(label)
            # img = cv2.imread(self.data_path + "/" + midname,cv2.IMREAD_GRAYSCALE)
            # label = cv2.imread(self.label_path + "/" + midname,cv2.IMREAD_GRAYSCALE)
            # img = np.array([img])
            # label = np.array([label])
            imgdatas[i] = img
            imglabels[i] = label
            if i % 100 == 0:
                print('Done: {0}/{1} images'.format(i, len(imgs)))
            i += 1
        print('loading done')
        np.save(self.npy_path + '/imgs_train.npy', imgdatas)
        np.save(self.npy_path + '/imgs_mask_train.npy', imglabels)
        print('Saving to .npy files done.')

    # 创建测试数据
    def create_test_data(self):
        i = 0
        print('-' * 30)
        print('Creating test images...')
        print('-' * 30)
        imgs = glob.glob(self.test_path + "/*." + self.img_type)
        print(len(imgs))
        imgdatas = np.ndarray((len(imgs), self.out_rows, self.out_cols, 3), dtype=np.uint8)
        for imgname in imgs:
            midname = imgname[imgname.rindex("\\") + 1:]
            img = load_img(self.test_path + "/" + midname, color_mode='rgb')
            img = img_to_array(img)
            # img = cv2.imread(self.test_path + "/" + midname,cv2.IMREAD_GRAYSCALE)
            # img = np.array([img])
            imgdatas[i] = img
            i += 1
        print('loading done')
        np.save(self.npy_path + '/imgs_test.npy', imgdatas)
        print('Saving to imgs_test.npy files done.')

    # 加载训练图片与mask
    def load_train_data(self):
        print('-' * 30)
        print('load train images...')
        print('-' * 30)
        imgs_train = np.load(self.npy_path + "/imgs_train.npy")
        imgs_mask_train = np.load(self.npy_path + "/imgs_mask_train.npy")
        imgs_train = imgs_train.astype('float32')
        imgs_mask_train = imgs_mask_train.astype('float32')
        imgs_train /= 255
        # mean = imgs_train.mean(axis=0)
        # imgs_train -= mean
        imgs_mask_train /= 255
        # 做一个阈值处理，输出的概率值大于0.5的就认为是对象，否则认为是背景
        imgs_mask_train[imgs_mask_train > 0.5] = 1
        imgs_mask_train[imgs_mask_train <= 0.5] = 0
        return imgs_train, imgs_mask_train

    # 加载测试图片
    def load_test_data(self):
        print('-' * 30)
        print('load test images...')
        print('-' * 30)
        imgs_test = np.load(self.npy_path + "/imgs_test.npy")
        imgs_test = imgs_test.astype('float32')
        imgs_test /= 255
        # mean = imgs_test.mean(axis=0)
        # imgs_test -= mean
        return imgs_test

IMAGE_ORDERING = 'channels_last'
def VGG16(n_classes,importModel = None):

    image_input = Input(shape = (512,512,3))
    #block1
    x = Conv2D(64,(3,3),activation = 'relu',padding = 'same',name = 'block1_conv1')(image_input)
    x = Conv2D(64,(3,3),activation = 'relu',padding = 'same', name = 'block1_conv2')(x)
    f1 = x

    x = MaxPooling2D((2,2), strides = (2,2), name = 'block1_pool')(x)
    x = Conv2D(128,(3,3),activation = 'relu',padding = 'same',name = 'block2_conv1')(x)
    x = Conv2D(128,(3,3),activation = 'relu',padding = 'same',name = 'block2_conv2')(x)
    f2 = x

    x = MaxPooling2D((2,2),strides = (2,2),name = 'block2_pool')(x)
    x = Conv2D(256,(3,3),activation = 'relu',padding = 'same',name = 'block3_conv1')(x)
    x = Conv2D(256,(3,3),activation = 'relu',padding = 'same',name = 'block3_conv2')(x)
    x = Conv2D(256,(3,3),activation = 'relu',padding = 'same',name = 'block3_conv3')(x)
    f3 = x

    x = MaxPooling2D((2,2),strides = (2,2),name = 'block3_pool')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block4_conv1')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block4_conv2')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block4_conv3')(x)
    f4 = x

    x = MaxPooling2D((2,2),strides = (2,2),name = 'block4_pool')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block5_conv1')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block5_conv2')(x)
    x = Conv2D(512,(3,3),activation = 'relu',padding = 'same', name = 'block5_conv3')(x)
    f5 = x

    x = MaxPooling2D((2,2),strides = (2,2),name = 'block5_pool')(x)


    # x = Flatten(name='flatten')(x)
    # x = Dense(4096, activation='relu', name='fc1')(x)
    # x = Dense(4096, activation='relu', name='fc2')(x)
    # x = Dense(1000, activation='softmax', name='predictions')(x)
    vgg = Model(image_input, x, name='vgg16')
    vgg.load_weights("./vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

    levels = [f1, f2, f3, f4, f5]

    o = f5
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(512, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    o = (concatenate([o, f4], axis=3))
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(256, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    o = (concatenate([o, f3], axis=3))
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(128, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    o = (concatenate([o, f2], axis=3))
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    o = (concatenate([o, f1], axis=3))
    o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(32, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = Conv2D(n_classes, (3, 3), padding='same', data_format=IMAGE_ORDERING)(o)
    o= Conv2D(1, 1, activation='sigmoid')(o)
    
    model = Model(image_input, o)
    
    return model

def load_data():
    mydata = dataProcess(512, 512)
    imgs_train, imgs_mask_train = mydata.load_train_data()
    imgs_test = mydata.load_test_data()
    return imgs_train, imgs_mask_train, imgs_test

if __name__ == "__main__":
    model = VGG16(2,'vgg16_weights.h5')
#     print(model.summary())    
    imgs_train, imgs_mask_train, imgs_test=load_data()
    model = VGG16(2,'vgg16_weights.h5')
    # print(model.summary())
    model_checkpoint = ModelCheckpoint('my_VGG_unet.hdf5', monitor='loss', verbose=1, save_best_only=True)
    model.compile(optimizer=Adam(lr = 1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(imgs_train, imgs_mask_train, batch_size=4, 
              nb_epoch=10, verbose=1, validation_split=0.2,
              shuffle=True, callbacks=[model_checkpoint,
            TensorBoard(log_dir=r"./mytensotboard_vggunet", 
                        update_freq="batch")])





------------------------------
load train images...
------------------------------
------------------------------
load test images...
------------------------------


D:\anaconda_64anzhuang\lib\site-packages\ipykernel_launcher.py:203: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1 samples, validate on 1 samples
Epoch 1/10
1/1 [==============================] - 25s 25s/step - loss: 0.7671 - acc: 0.5234 - val_loss: 0.9030 - val_acc: 0.2855

Epoch 00001: loss improved from inf to 0.76710, saving model to my_VGG_unet.hdf5
Epoch 2/10
1/1 [==============================] - 22s 22s/step - loss: 0.7005 - acc: 0.5669 - val_loss: 0.8588 - val_acc: 0.2862

Epoch 00002: loss improved from 0.76710 to 0.70050, saving model to my_VGG_unet.hdf5
Epoch 3/10
1/1 [==============================] - 23s 23s/step - loss: 0.6684 - acc: 0.6087 - val_loss: 0.8492 - val_acc: 0.3575

Epoch 00003: loss improved from 0.70050 to 0.66842, saving model to my_VGG_unet.hdf5
Epoch 4/10
1/1 [==============================] - 25s 25s/step - loss: 0.6494 - acc: 0.6566 - val_loss: 0.8539 - val_acc: 0.4229

Epoch 00004: loss improved from 0.66842 to 0.64936, saving model to my_VGG_unet.hdf5
Epoch 5/10
1/1 [==============================] - 23s 23s/step - loss: 0.6377 - acc: 0.6827 - val_loss